In [ ]:
scd = {}
scd['ACR_49'] = [1]

In [ ]:
from acr.utils import SOM_BLUE, NNXR_GRAY, CTRL_BLUE

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'control'
MAIN_COLOR = CTRL_BLUE

In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import warnings
import pingouin as pg
from scipy.stats import shapiro, normaltest

warnings.filterwarnings('ignore')
probe_ord = ['NNXr', 'NNXo']
hue_ord = [NNXR_GRAY, MAIN_COLOR]

from acr.plots import pub, lrg

plt.style.use('fast')
pub()
plt.rcParams['xtick.bottom'] = False
#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import *
data_agg = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from data_agg import *

In [ ]:
subjects, exps = get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
subjects,  exps = pub_utils.remove_subject('ACR_38', subjects, exps)
#subjects,  exps = pub_utils.remove_subject('ACR_42', subjects, exps)

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, extra_rebounds=False)

In [ ]:
#load all raw bp dfs
bp_dfs = {}
for sub, exp in zip(subjects, exps):
    bp = load_raw_bp_df(sub, exp)
    #bp = acr.hypnogram_utils.label_df_with_hypno_conditions(bp, hyp_dicts[sub])
    #bp = data_agg._label_12_hr_baseline(bp)
    bp_dfs[sub] = bp

In [ ]:
reldfs = []
for subject in subjects:
    print(subject)
    subdf = bp_dfs[subject]
    subdf = acr.hypnogram_utils.label_df_with_hypno_conditions(subdf, hyp_dicts[subject])
    print('Done conditions')
    subdf = acr.hypnogram_utils.label_df_with_states(subdf, full_hyps[subject])
    subdf = acr.hypnogram_utils.label_df_with_full_bl(subdf)
    subdf = make_raw_bp_df_relative_to_baseline(subdf, col_to_use='full_bl', value_to_use='True', state_to_use='NREM', method='mean')
    print('Done states/bl')
    reldfs.append(subdf)
relbp = pl.concat(reldfs)

In [ ]:
# drop the known bad channels
relbp = drop_sub_channels(relbp, scd)
rebbp = relbp.filter(pl.col('condition') == 'rebound')

In [ ]:
plt.rcdefaults()
# quick data check to make sure all remaining channels are good
g = sns.catplot(data=relbp.filter(pl.col('band') == 'delta').filter(pl.col('condition')=='rebound').to_pandas(), 
                x='channel', 
                y='bandpower_rel', 
                hue='store', 
                kind='box',
                row='subject',
                row_order=subjects,
                showfliers=False,
                showmeans=True, meanprops={"marker": "_", 'markerfacecolor':'gold','markeredgecolor':'gold','markersize':'24', "markeredgewidth": "3"},
                height=10,
                aspect=3,
                hue_order=probe_ord, 
                palette=[NNXR_GRAY, SOM_BLUE]
                )

for ax in g.axes.flat:
    ax.set_ylim(0, 5)

In [ ]:
reb_avgs = rebbp.group_by(['subject', 'store', 'band']).mean().to_pandas()
reb_avgs = sub_regions_to_df(reb_avgs)

# SWA, simple average of all channels

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], fsize=(3.5, 4))

ax.set_ylim(1, 1.8) # y limits
ax.set_xlim(0.30, 0.70) # x limits
ax.set_yticks([1, 1.4, 1.8]) # y ticks
ax.set_xticklabels(['Contra. Control', 'Optrode'])
#ax.set_yticks([1.25, 1.75], minor=True) # minor y ticks

#plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)


In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
if write:
    # ===== Write Stats =====
    stats_name = f'REB-SWA__{SUBJECT_TYPE.upper()}__{SUBJECT_GROUP}__{CHANNEL_GROUP}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}__NORMALIZED'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

nnxr_norm = nnxr/nnxr
nnxo_norm = nnxo/nnxr

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr_norm, nnxo_norm, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True, fsize=(3.5, 4), lineplot_width=3.5)
print(ax.get_ylim())

ax.set_ylim(0.98, 1.16) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([1, 1.1]) # y ticks
ax.set_xticklabels(['Contra. Control', 'Optrode'])
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'FRONTAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "frontal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=False, fsize=(3.5, 4), lineplot_width=3.5)
print(ax.get_ylim())
ax.set_ylim(1.65, 1.75) # y limits
ax.set_xlim(0.30, 0.70) # x limits
ax.set_yticks([1.65, 1.75]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
#agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
if write:
    # ===== Write Stats =====
    stats_name = f'REB-SWA__{SUBJECT_TYPE.upper()}__{SUBJECT_GROUP}__{CHANNEL_GROUP}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
    
stats

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'FRONTAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}__NORMALIZED'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "frontal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

nnxr_norm = nnxr/nnxr
nnxo_norm = nnxo/nnxr

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr_norm, nnxo_norm, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True, fsize=(3.5, 4), lineplot_width=3.5)


ax.set_ylim(0.9, 1.01) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([0.9, 1]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'PARIETAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "parietal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=False, fsize=(3.5, 4), lineplot_width=3.5)
print(ax.get_ylim())
ax.set_ylim(1, 1.8) # y limits
ax.set_xlim(0.30, 0.70) # x limits
ax.set_yticks([1, 1.4, 1.8]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
#agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')
stats = pg.ttest(nnxr, nnxo, paired=True)
hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')
if write:
    # ===== Write Stats =====
    stats_name = f'REB-SWA__{SUBJECT_TYPE.upper()}__{SUBJECT_GROUP}__{CHANNEL_GROUP}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pub_utils.write_source_data(source_data, stats_name)
stats

In [ ]:
# DEFINE THE DATA
# --------------------------------------------
BAND = 'delta'
SUBJECT_GROUP = 'PARIETAL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = f'{BAND}_rebound_{MAIN_EXP}__NORMALIZED'
figure_name = f'{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{FIG_ID}.png'

#---------------- filter the data ----------------#
data = reb_avgs.query('band == @BAND').query('region == "parietal"')
data = data.sort_values(by=['subject', 'store'])
nnxr = data.query('store == "NNXr"')['bandpower_rel'].values
nnxo = data.query('store == "NNXo"')['bandpower_rel'].values

nnxr_norm = nnxr/nnxr
nnxo_norm = nnxo/nnxr

# PLOT AND SAVE
# ---------------------------------------------
f, ax = acr.plots.gen_paired_boxplot(nnxr_norm, nnxo_norm, colors=[NNXR_GRAY, MAIN_COLOR], one_sided=True, fsize=(3.5, 4), lineplot_width=3.5)
print(ax.get_ylim())
ax.set_ylim(0.95, 1.16) # y limits
ax.set_xlim(0.35, 0.70) # x limits
ax.set_yticks([1, 1.1]) # y ticks

plt.tight_layout(pad=0.2)
plt.savefig(f'{PAPER_FIGURE_ROOT}/swa/{figure_name}', dpi=600, bbox_inches='tight', transparent=True)


# Full Spectrograms

In [ ]:
reb_spgs = {}
for subject, exp in zip(subjects, exps):
    if subject in scd:
        to_drop = scd[subject]
    else:
        to_drop=None
    reb_spgs[subject] = create_reb_spg(subject, exp, drop_chans=to_drop)
full_df = pd.concat(reb_spgs.values())
full_means = full_df.groupby(['store', 'frequency']).mean(numeric_only=True).reset_index()

In [ ]:
from statsmodels.stats.multitest import multipletests
# statistical-tests for all frequencies
t_tests = {}
freqs = []
p_vals = []
for freq in full_df['frequency'].unique():
    freqs.append(freq)
    freq_df = full_df.loc[full_df['frequency'] == freq].sort_values(['store', 'subject'])
    nnxr = freq_df.query('store == "NNXr"')['power_rel'].values
    nnxo = freq_df.query('store == "NNXo"')['power_rel'].values
    
    # will use wilcoxon across all frequencies to avoid differing normality assumptions across different frequencies, etc
    stat = pg.wilcoxon(nnxr, nnxo)
    p_vals.append(stat['p-val'][0])

In [ ]:
# Correct for multiple Comparisons using Benjamini–Hochberg False discovery rate correction 
rej, p_fdr, _, _ = multipletests(p_vals, alpha=0.05, method='fdr_bh')

for p_cor, freq in zip(p_fdr, freqs):
    if p_cor<0.05:
        print(round(freq, 1), round(p_cor, 3))
    #print(round(freq, 1), round(p_cor, 3))


In [ ]:
# NO significance

In [ ]:
# =============================
# ===== Write Source Data =====
# =============================
dfs = []
for freq in full_df['frequency'].unique():
    freqs.append(freq)
    freq_df = full_df.loc[full_df['frequency'] == freq].sort_values(['store', 'subject'])
    nnxr = freq_df.query('store == "NNXr"')['power_rel'].values
    nnxo = freq_df.query('store == "NNXo"')['power_rel'].values
    f = round(freq, 1)
    fdf = pd.DataFrame({'freq_bin': f, 'contra_control_power': nnxr, 'optrode_power': nnxo, 'subject': np.arange(len(nnxr))})
    dfs.append(fdf)
spg_source_dat = pd.concat(dfs)
pub_utils.write_source_data(spg_source_dat, 'CTRL_full_spg_source_data')

In [ ]:
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = 'spg_rebound'

fig_path = f'{PAPER_FIGURE_ROOT}/spectral/{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{MAIN_EXP}--{FIG_ID}.png'
# --------------------------------------------------------------------------------------------------

plt.rcdefaults()  # Reset to matplotlib defaults
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True

f, ax = plt.subplots(1, 1, figsize=(4.5, 3))
sns.lineplot(data=full_df, x='frequency', y='power_rel', hue='store', ax=ax, palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, errorbar='se', lw=4)

ax.set_xlim(0, 20)  
ax.set_ylim(0.8, 1.6)
ax.set_yticks([0.8, 1, 1.2, 1.4, 1.6])
ax.set_xticks([0, 5, 10, 15, 20])

#remove the legend
ax.legend_.remove()
#ax.axhline(y=1.59, xmin=0.09, xmax=0.15)
#ax.axvspan(0.3, 2.4, ymin=0.985, ymax=1.0)
#ax.axvspan(2.6, 3.5, ymin=0.985, ymax=1.0)

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
SUBJECT_GROUP = 'ALL_SUBJECTS'
CHANNEL_GROUP = 'ALL_CHANNELS'
FIG_ID = 'spg_rebound__SIGNIFICANCE_MARKERS'

fig_path = f'{PAPER_FIGURE_ROOT}/spectral/{SUBJECT_TYPE.upper()}--{SUBJECT_GROUP}--{CHANNEL_GROUP}--{MAIN_EXP}--{FIG_ID}.svg'
# --------------------------------------------------------------------------------------------------

plt.rcdefaults()  # Reset to matplotlib defaults
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True

f, ax = plt.subplots(1, 1, figsize=(4.5, 3))
sns.lineplot(data=full_df, x='frequency', y='power_rel', hue='store', ax=ax, palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, errorbar='se', lw=4)

ax.set_xlim(0, 20)  
ax.set_ylim(0.8, 1.6)
ax.set_yticks([0.8, 1, 1.2, 1.4, 1.6])
ax.set_xticks([0, 5, 10, 15, 20])

#remove the legend
ax.legend_.remove()

ax.hlines(1.5, 0.5, 3)

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

# FUll SPG

In [ ]:
spgs = {}
for subject, exp in zip(subjects, exps):
    spgs[subject] = data_agg.load_raw_spg(subject, exp)

In [ ]:
subject = 'ACR_21'
spg = spgs[subject]

In [ ]:
fspgs = []
for subject in spgs:
    fspgs.append(data_agg.get_bl_reb_spg_df(spgs[subject], full_hyps, hyp_dicts, subject))

fspg = pl.concat(fspgs)

In [ ]:
def get_relative_rebound_band(fullspg, flow, fhigh):
    band_spg = fullspg.filter(pl.col('frequency') > flow).filter(pl.col('frequency') < fhigh)
    band_spg = band_spg.group_by(['subject', 'datetime', 'store', 'channel', 'state', 'condition']).agg(pl.col('power').sum()).sort('subject','store', 'channel')
    band_spg = data_agg.relativize_df(band_spg, 'condition', 'full_bl', col_to_relativize='power', on=['subject','store', 'channel'])
    rebband = band_spg.cdn('rebound')
    return rebband

In [ ]:
delta_rebound = get_relative_rebound_band(fspg, 0.4, 4)

In [ ]:
bm = delta_rebound.group_by(['subject', 'store', 'channel']).agg(pl.col('power_rel').mean()).sort('subject', 'store')

In [ ]:
nnxo = bm.prb('NNXo')['power_rel'].to_numpy()
nnxr = bm.prb('NNXr')['power_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo)